# Performance Report and Analysis

This notebook provides detailed performance analysis and risk metrics.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.feature_engineer import FeatureEngineer
from src.regime_detection.gmm_detector import GMMDetector
from src.strategies.trend_following import TrendFollowingStrategy
from src.strategies.backtester import Backtester

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Strategy Results

In [ ]:
# Prepare data
loader = DataLoader()
raw_data = loader.load_data('SPY', start_date='2015-01-01', end_date='2023-12-31')

preprocessor = DataPreprocessor()
clean_data = preprocessor.clean_data(raw_data)

engineer = FeatureEngineer()
features = engineer.create_features(clean_data)
regime_features = engineer.extract_regime_features(features)

detector = GMMDetector(n_regimes=3, random_state=42)
detector.fit(regime_features)
regimes = detector.predict(regime_features)

# Run backtest
strategy = TrendFollowingStrategy()
backtester = Backtester(initial_capital=100000)
results = backtester.run(strategy, features, regime_labels=regimes)

print("Results loaded successfully")

## 2. Drawdown Analysis

In [ ]:
# Calculate drawdown
equity = results['equity_curve']
running_max = equity.expanding().max()
drawdown = (equity - running_max) / running_max

# Plot drawdown
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Equity curve
axes[0].plot(equity.index, equity, linewidth=2, label='Portfolio Value')
axes[0].plot(running_max.index, running_max, '--', linewidth=1.5, 
             alpha=0.7, color='red', label='Peak Value')
axes[0].set_title('Equity Curve with Peak', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Value ($)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
axes[1].fill_between(drawdown.index, drawdown, 0, alpha=0.5, color='red')
axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Drawdown (%)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Drawdown statistics
print("\nDrawdown Statistics:")
print(f"Max Drawdown: {drawdown.min():.2%}")
print(f"Avg Drawdown: {drawdown[drawdown < 0].mean():.2%}")

# Find worst drawdown period
worst_dd_idx = drawdown.idxmin()
print(f"Worst Drawdown Date: {worst_dd_idx}")

## 3. Returns Analysis

In [ ]:
# Get strategy returns
returns = results['returns']
market_returns = features['returns']

# Plot returns distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(returns, bins=100, alpha=0.7, label='Strategy', edgecolor='black')
axes[0].hist(market_returns, bins=100, alpha=0.5, label='Market', edgecolor='black')
axes[0].axvline(x=returns.mean(), color='r', linestyle='--', 
                label=f'Strategy Mean: {returns.mean():.4f}')
axes[0].axvline(x=market_returns.mean(), color='b', linestyle='--',
                label=f'Market Mean: {market_returns.mean():.4f}')
axes[0].set_title('Returns Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Return', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(returns.dropna(), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Returns statistics
print("\nReturns Statistics:")
print(f"Mean:             {returns.mean():.4f}")
print(f"Median:           {returns.median():.4f}")
print(f"Std Dev:          {returns.std():.4f}")
print(f"Skewness:         {returns.skew():.4f}")
print(f"Kurtosis:         {returns.kurtosis():.4f}")
print(f"Min:              {returns.min():.4f}")
print(f"Max:              {returns.max():.4f}")

## 4. Rolling Performance Metrics

In [ ]:
# Calculate rolling metrics
window = 252  # 1 year

rolling_sharpe = (returns.rolling(window).mean() / returns.rolling(window).std()) * np.sqrt(252)
rolling_vol = returns.rolling(window).std() * np.sqrt(252)

# Plot rolling metrics
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Rolling Sharpe
axes[0].plot(rolling_sharpe.index, rolling_sharpe, linewidth=2)
axes[0].axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Sharpe = 1')
axes[0].set_title('Rolling Sharpe Ratio (252-day)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Sharpe Ratio', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Rolling Volatility
axes[1].plot(rolling_vol.index, rolling_vol, linewidth=2, color='orange')
axes[1].set_title('Rolling Volatility (252-day)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Annualized Volatility', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Trade Analysis

In [ ]:
# Get trades
trades = results['trades']

if len(trades) > 0:
    print(f"Total Trades: {len(trades)}")
    print(f"\nFirst 10 trades:")
    display(trades.head(10))
    
    # Analyze trade performance
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Trade returns distribution
    axes[0,0].hist(trades['return'], bins=50, edgecolor='black', alpha=0.7)
    axes[0,0].axvline(x=0, color='r', linestyle='--')
    axes[0,0].set_title('Trade Returns Distribution', fontweight='bold')
    axes[0,0].set_xlabel('Return')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].grid(True, alpha=0.3)
    
    # Trade returns over time
    axes[0,1].scatter(range(len(trades)), trades['return'], alpha=0.6)
    axes[0,1].axhline(y=0, color='r', linestyle='--')
    axes[0,1].set_title('Trade Returns Over Time', fontweight='bold')
    axes[0,1].set_xlabel('Trade Number')
    axes[0,1].set_ylabel('Return')
    axes[0,1].grid(True, alpha=0.3)
    
    # Holding period distribution
    axes[1,0].hist(trades['holding_period'], bins=30, edgecolor='black', alpha=0.7)
    axes[1,0].set_title('Holding Period Distribution', fontweight='bold')
    axes[1,0].set_xlabel('Days')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].grid(True, alpha=0.3)
    
    # Long vs Short performance
    long_trades = trades[trades['direction'] == 'Long']
    short_trades = trades[trades['direction'] == 'Short']
    
    direction_stats = pd.DataFrame({
        'Long': [len(long_trades), long_trades['return'].mean(), 
                 (long_trades['return'] > 0).sum() / len(long_trades) if len(long_trades) > 0 else 0],
        'Short': [len(short_trades), short_trades['return'].mean(),
                  (short_trades['return'] > 0).sum() / len(short_trades) if len(short_trades) > 0 else 0]
    }, index=['Count', 'Avg Return', 'Win Rate'])
    
    axes[1,1].axis('tight')
    axes[1,1].axis('off')
    table = axes[1,1].table(cellText=direction_stats.values, 
                            rowLabels=direction_stats.index,
                            colLabels=direction_stats.columns,
                            cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    axes[1,1].set_title('Long vs Short Performance', fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades executed")

## 6. Risk Metrics Summary

In [ ]:
# Compile risk metrics
metrics = results['metrics']

risk_summary = pd.DataFrame({
    'Metric': [
        'Total Return',
        'CAGR',
        'Sharpe Ratio',
        'Sortino Ratio',
        'Calmar Ratio',
        'Max Drawdown',
        'Annual Volatility',
        'Win Rate',
        'Profit Factor',
        'Total Trades'
    ],
    'Value': [
        f"{metrics['total_return']:.2%}",
        f"{metrics['cagr']:.2%}",
        f"{metrics['sharpe_ratio']:.2f}",
        f"{metrics['sortino_ratio']:.2f}",
        f"{metrics['calmar_ratio']:.2f}",
        f"{metrics['max_drawdown']:.2%}",
        f"{metrics['annual_volatility']:.2%}",
        f"{metrics['win_rate']:.2%}",
        f"{metrics['profit_factor']:.2f}",
        f"{metrics['n_trades']}"
    ]
})

print("\n" + "="*60)
print("RISK METRICS SUMMARY")
print("="*60)
display(risk_summary)

# Visualize key metrics
fig, ax = plt.subplots(figsize=(10, 6))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=risk_summary.values,
                colLabels=risk_summary.columns,
                cellLoc='left',
                loc='center',
                colWidths=[0.6, 0.4])

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)

# Color header
for i in range(len(risk_summary.columns)):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.title('Performance Summary', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## 7. Monte Carlo Simulation

In [ ]:
# Run Monte Carlo simulation
n_simulations = 1000
n_periods = 252  # 1 year

# Use historical return distribution
mean_return = returns.mean()
std_return = returns.std()

# Simulate paths
np.random.seed(42)
simulations = np.zeros((n_periods, n_simulations))

for i in range(n_simulations):
    returns_sim = np.random.normal(mean_return, std_return, n_periods)
    simulations[:, i] = (1 + returns_sim).cumprod()

# Plot simulation results
plt.figure(figsize=(14, 7))

# Plot all paths (with transparency)
plt.plot(simulations, color='blue', alpha=0.01, linewidth=0.5)

# Plot percentiles
percentiles = [5, 25, 50, 75, 95]
for p in percentiles:
    percentile_path = np.percentile(simulations, p, axis=1)
    plt.plot(percentile_path, linewidth=2, label=f'{p}th percentile')

plt.title('Monte Carlo Simulation (1000 paths, 1 year)', fontsize=14, fontweight='bold')
plt.xlabel('Days', fontsize=12)
plt.ylabel('Portfolio Value (Normalized)', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Final value distribution
final_values = simulations[-1, :]
plt.figure(figsize=(10, 6))
plt.hist(final_values, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=np.median(final_values), color='r', linestyle='--', 
            linewidth=2, label=f'Median: {np.median(final_values):.2f}')
plt.title('Distribution of Final Portfolio Values', fontsize=14, fontweight='bold')
plt.xlabel('Final Value (Normalized)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMonte Carlo Results (1 year):")
print(f"Expected Value: {np.mean(final_values):.2f}")
print(f"Median Value: {np.median(final_values):.2f}")
print(f"5th Percentile: {np.percentile(final_values, 5):.2f}")
print(f"95th Percentile: {np.percentile(final_values, 95):.2f}")
print(f"Probability of Profit: {(final_values > 1).sum() / n_simulations:.2%}")

## Conclusion

This notebook provides comprehensive performance analysis including:
- Drawdown analysis
- Returns distribution
- Rolling performance metrics
- Trade-level analysis
- Risk metrics summary
- Monte Carlo projections